# Check dataflow:
- training

In [1]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

sys.path.insert(0, (Path(".").resolve().parent / "tensorpack-FasterRCNN").as_posix())

In [2]:
from data import get_train_dataflow, cfg, COCODetection, logger, DataFromList, CustomResize, imgaug, cv2
from data import box_to_point8, point8_to_box, get_multilevel_rpn_anchor_input, itertools, copy
from data import np_area, np_iou, MalformedData, get_rpn_anchor_input

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
cfg.DATA.BASEDIR="../input/as_mscoco"
cfg.MODE_MASK = False
cfg.DATA.TRAIN = ['test', 'val_0.7', ]

In [8]:
from config import finalize_configs

In [9]:
_ = finalize_configs(is_training=True)

[0708 16:22:54 @config.py:190] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCK': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': True,
              'WEIGHTS': ''},
 'DATA': {'BASEDIR': '../input/as_mscoco',
          'CLASS_NAMES': ['BG',
                          'Tortoise',
                          'Container',
                          'Magpie',
                          'Sea turtle',
                          'Football',
                          'Ambulance',
                          'Ladder',
                          'Toothbrush',
                          'Syringe',
                          'Sink',
                          'Toy',
                          'Organ',
                          'Cassette deck',
                          'Apple',
                          'Human eye',
                          'Cosmetics',
                      

In [10]:
imgs = COCODetection.load_many(cfg.DATA.BASEDIR, cfg.DATA.TRAIN, add_gt=True, add_mask=cfg.MODE_MASK)
num = len(imgs)
imgs = list(filter(lambda img: len(img['boxes'][img['is_crowd'] == 0]) > 0, imgs))    # log invalid training
logger.info("Filtered {} images which contain no groudtruth boxes. Total #images for training: {}"
            .format(num - len(imgs), len(imgs)))

ds = DataFromList(imgs, shuffle=True)

aug = imgaug.AugmentorList(
    [CustomResize(cfg.PREPROC.SHORT_EDGE_SIZE, cfg.PREPROC.MAX_SIZE),
     imgaug.Flip(horiz=True)])

def preprocess(img):
    fname, boxes, klass, is_crowd = img['file_name'], img['boxes'], img['class'], img['is_crowd']
    boxes = np.copy(boxes)
    im = cv2.imread(fname, cv2.IMREAD_COLOR)
    assert im is not None, fname
    im = im.astype('float32')
    # assume floatbox as input
    assert boxes.dtype == np.float32, "Loader has to return floating point boxes!"

    # augmentation:
    im, params = aug.augment_return_params(im)
    points = box_to_point8(boxes)
    points = aug.augment_coords(points, params)
    boxes = point8_to_box(points)
    assert np.min(np_area(boxes)) > 0, "Some boxes have zero area!"

    # rpn anchor:
    try:
        if cfg.MODE_FPN:
            multilevel_anchor_inputs = get_multilevel_rpn_anchor_input(im, boxes, is_crowd)
            anchor_inputs = itertools.chain.from_iterable(multilevel_anchor_inputs)
        else:
            # anchor_labels, anchor_boxes
            anchor_inputs = get_rpn_anchor_input(im, boxes, is_crowd)
            assert len(anchor_inputs) == 2

        boxes = boxes[is_crowd == 0]    # skip crowd boxes in training target
        klass = klass[is_crowd == 0]
        if not len(boxes):
            raise MalformedData("No valid gt_boxes!")
    except MalformedData as e:
        log_once("Input {} is filtered for training: {}".format(fname, str(e)), 'warn')
        return None

    ret = [im] + list(anchor_inputs) + [boxes, klass]

    if cfg.MODE_MASK:
        # augmentation will modify the polys in-place
        segmentation = copy.deepcopy(img['segmentation'])
        segmentation = [segmentation[k] for k in range(len(segmentation)) if not is_crowd[k]]
        assert len(segmentation) == len(boxes)

        # Apply augmentation on polygon coordinates.
        # And produce one image-sized binary mask per box.
        masks = []
        for polys in segmentation:
            polys = [aug.augment_coords(p, params) for p in polys]
            masks.append(segmentation_to_mask(polys, im.shape[0], im.shape[1]))
        masks = np.asarray(masks, dtype='uint8')    # values in {0, 1}
        ret.append(masks)

        # from viz import draw_annotation, draw_mask
        # viz = draw_annotation(im, boxes, klass)
        # for mask in masks:
        #     viz = draw_mask(viz, mask)
        # tpviz.interactive_imshow(viz)
    return ret


loading annotations into memory...
Done (t=3.45s)
creating index...
index created!
[0708 16:23:05 @coco.py:78] Instances loaded from ../input/as_mscoco/annotations/test.json.
[0708 16:23:22 @timer.py:44] Load Groundtruth Boxes for test finished, time:16.9761sec.
loading annotations into memory...
Done (t=1.91s)
creating index...
index created!
[0708 16:23:24 @coco.py:78] Instances loaded from ../input/as_mscoco/annotations/val_0.7.json.
[0708 16:23:29 @timer.py:44] Load Groundtruth Boxes for val_0.7 finished, time:4.1116sec.
[0708 16:23:29 @<ipython-input-10-f757234eb849>:5] Filtered 13007 images which contain no groudtruth boxes. Total #images for training: 120089


In [11]:
ds.reset_state()

In [16]:
for dp in ds.get_data():
    if 'val' in dp['file_name']:
        break
dp

{'boxes': array([[   0.,    0., 1023.,  826.],
        [ 111.,   36.,  730.,  693.],
        [ 413.,  451., 1024.,  826.],
        [ 893.,  318., 1024.,  826.]], dtype=float32),
 'class': array([334, 456, 456, 456], dtype=int32),
 'file_name': '/home/fast_storage/OpenImagesDataset/validation/7d77fa1b81e24547.jpg',
 'height': 826,
 'id': '7d77fa1b81e24547',
 'is_crowd': array([1, 0, 0, 0], dtype=int8),
 'width': 1024}

In [17]:
ret = preprocess(dp)

In [18]:
type(ret), len(ret), ret[0].shape

(list, 5, (800, 992, 3))

In [19]:
ret[1].shape, ret[2].shape, ret[3].shape, ret[4].shape

((84, 84, 15), (84, 84, 15, 4), (3, 4), (3,))

Eval dataset

In [27]:
cfg.DATA.VAL = 'val_0.3'
_ = finalize_configs(is_training=False)

[0708 17:13:40 @config.py:190] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCK': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': True,
              'WEIGHTS': ''},
 'DATA': {'BASEDIR': '../input/as_mscoco',
          'CLASS_NAMES': ['BG',
                          'Tortoise',
                          'Container',
                          'Magpie',
                          'Sea turtle',
                          'Football',
                          'Ambulance',
                          'Ladder',
                          'Toothbrush',
                          'Syringe',
                          'Sink',
                          'Toy',
                          'Organ',
                          'Cassette deck',
                          'Apple',
                          'Human eye',
                          'Cosmetics',
                      

In [28]:
from data import get_eval_dataflow

In [29]:
ds = get_eval_dataflow()

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
[0708 17:13:53 @coco.py:78] Instances loaded from ../input/as_mscoco/annotations/val_0.3.json.
[0708 17:13:53 @timer.py:44] Load Groundtruth Boxes for val_0.3 finished, time:0.0644sec.
PrefetchDataZMQ successfully cleaned-up.


In [30]:
ds.reset_state()

for dp in ds.get_data():
    break

type(dp), len(dp), type(dp[0]), type(dp[1])

(list, 2, numpy.ndarray, str)

test dump cfg

In [11]:
conf_dict = cfg.to_dict()

import json

with open("config.json", "w") as h:
    json.dump(conf_dict, h)

In [12]:
rm config.json